In [1]:
%pylab tk

Populating the interactive namespace from numpy and matplotlib


In [2]:
from scipy.integrate import odeint
from __future__ import division
import time
import matplotlib.animation as animation

In [3]:
species = 5
# protein, capture, analyte, control
prot_i, cap_i, anal_i, prot_cap_i, prot_anal_i = range(0,species)

def f(y, t, plate_flow, Ka_capture, k_capture_on, Ka_analyte, k_analyte_on):
    #get the input vector and reshape it into an interpretable array
    ar_y = y.reshape(-1, species)
    #array that we'll return with derivatives
    out = zeros(ar_y.shape)
    
    #calculate rates
    k_capture_off = k_capture_on / Ka_capture
    k_analyte_off = k_analyte_on / Ka_analyte
    
    #get vectors for the concentration of each species
    prot = ar_y[:,prot_i]
    cap = ar_y[:,cap_i]
    anal = ar_y[:,anal_i]
    prot_cap = ar_y[:,prot_cap_i]
    prot_anal = ar_y[:,prot_anal_i]    
    
    #equilibrate each plate
    out[:,prot_i] = k_capture_off*prot_cap - k_capture_on*prot*cap + k_analyte_off*prot_anal - k_analyte_on*prot*anal
    out[:,anal_i] = k_analyte_off*prot_anal - k_analyte_on*prot*anal
    out[:,cap_i] = k_capture_off*prot_cap - k_capture_on*prot*cap
    out[:,prot_anal_i] = -(k_analyte_off*prot_anal - k_analyte_on*prot*anal)
    out[:,prot_cap_i] = -(k_capture_off*prot_cap - k_capture_on*prot*cap)  
    
    #flow between plates
    for key in [prot_i, anal_i, prot_anal_i]:
        additional = hstack((0, ar_y[:,key]))
        leaving = - plate_flow*ar_y[:,key]
        coming = plate_flow*additional[0:-1] 
        leaving[50:]=0
        coming[51:]=0
        out[:,key] = out[:,key] + leaving + coming
    
    return out.reshape(-1,1)[:,0]


npts=1000
def run_ode(protein_start=10**-10.0, anal_start=10**-10.0, cap_start=10**-5.0, \
            kanal=10**10.0, kcap=10**8.0, kinetic_factor = 100.0):
    #global parameters that we want to keep constant for all simulations
    width, length, thickness = 10.0, 50.0/2.0, 0.15#units of mm
    porosity = 0.7
    col_vol = porosity*width*length*thickness / 1000.0 #ml
    #print 'Column volume is %0.2f ul'%(col_vol*1000)

    flow = 0.001 #ml/min
    flow = flow/60.0 #ml/s
    plates = 51
    plate_vol = col_vol / plates
    plate_flow = flow/plate_vol
    #print 'Plate volume is %0.2f ul'%(plate_vol*1000)

    volumes = linspace(0, col_vol*2.0, npts)

    times = volumes / flow
    cvs = volumes / col_vol

    col = zeros((plates, species))
    col[0, prot_i] = protein_start
    col[0, anal_i] = anal_start
    col[40, cap_i] = cap_start

    linear_col = col.reshape(-1,1)

    start = time.time()
    result = odeint(f, linear_col[:,0], times, args=(plate_flow, kcap, kcap*kinetic_factor, kanal, kanal*kinetic_factor),\
                   printmessg=False, rtol=10**-18, atol=10**-18)
    print "Calculation took %0.2fs"%(time.time()-start)
    return result

#Test section

In [4]:
result = run_ode(protein_start = 10**-10.0, anal_start = 10**-8.0, cap_start = 10**-5.0)

Calculation took 2.84s


In [5]:
#animation of the setup
t = 0

cap = result[t,:].reshape(-1, species)[:,cap_i]
prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
prot = result[t,:].reshape(-1, species)[:,prot_i] 
anal = result[t,:].reshape(-1, species)[:,anal_i] 
prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i] 

fig = figure(1, figsize=(10, 5))
clf()

#plot(cap, 'o-', label='cap')
prot_cap_line, = plot(prot_cap*10**6, '-', label='prot-cap')
prot_line, = plot(prot*10**6, '-', label='prot')
#anal_line, = plot(anal*10**6, '-', label='anal')
prot_anal_line, = plot(prot_anal*10**6, label='prot_anal')
legend(loc='upper left')

#ylim((0, 0.0001))
grid()

def animate(i):
    i=int(i)
    prot_cap_line.set_ydata(result[i,:].reshape(-1, species)[:,prot_cap_i]*10**6)
    prot_line.set_ydata(result[i,:].reshape(-1, species)[:,prot_i] *10**6)
    #anal_line.set_ydata(result[i,:].reshape(-1, species)[:,anal_i] *10**6)
    prot_anal_line.set_ydata(result[i,:].reshape(-1, species)[:,prot_anal_i] *10**6)
    #return prot_cap_line,prot_line,

ani = animation.FuncAnimation(fig, animate, linspace(0, npts-1, 200),
    interval=10, blit=False, repeat=False)

show()

In [6]:
t = -1
cap = result[t,:].reshape(-1, species)[:,cap_i]
prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
prot = result[t,:].reshape(-1, species)[:,prot_i] 
anal = result[t,:].reshape(-1, species)[:,anal_i] 
prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i] 

allprot = prot_cap+prot+prot_anal

first_spot, second_spot = allprot[40], allprot[50]

print first_spot/second_spot

print log10(second_spot / first_spot)

8.80673088489
-0.944814725423


In [58]:
#check equilibrium
t = 100

cap = result[t,:].reshape(-1, species)[:,cap_i]
prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
prot = result[t,:].reshape(-1, species)[:,prot_i] 
anal = result[t,:].reshape(-1, species)[:,anal_i] 
prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i] 


figure(2)
clf()
plot(log10(prot_cap / (prot*cap)), 'o')
plot(log10(prot_anal / (prot*anal)), 'o')
draw()

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:13: RuntimeWarning: invalid value encountered in true_divide
C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:14: RuntimeWarning: invalid value encountered in true_divide


#Heat Maps

In [11]:
#2D heatmap
#analyte concentration
xstart, xstop, xdim = -10, -5, 10
#capture concentration
ystart, ystop, ydim = -6.5, -5.5, 10

pconc = -8.0

out = zeros((ydim, xdim))

for i,analpow in enumerate(linspace(xstart, xstop, xdim)):
    for j,cappow in enumerate(linspace(ystart, ystop, ydim)):
        print i,j
        result = run_ode(anal_start = 10**analpow, cap_start = 10**cappow, protein_start = 10**pconc)
        t = -1
        cap = result[t,:].reshape(-1, species)[:,cap_i]
        prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
        prot = result[t,:].reshape(-1, species)[:,prot_i] 
        anal = result[t,:].reshape(-1, species)[:,anal_i] 
        prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i] 
        allprot = prot_cap+prot+prot_anal
        first_spot, second_spot = abs(allprot[40]), abs(allprot[50])
        #print first_spot, second_spot
        out[j,i] = log10(second_spot/first_spot)
print 'done'

0 0
Calculation took 5.32s
0 1
Calculation took 4.99s
0 2
Calculation took 5.90s
0 3
Calculation took 5.08s
0 4
Calculation took 4.45s
0 5
Calculation took 4.53s
0 6
Calculation took 4.27s
0 7
Calculation took 4.34s
0 8
Calculation took 3.65s
0 9
Calculation took 3.52s
1 0
Calculation took 5.51s
1 1
Calculation took 6.19s
1 2
Calculation took 5.60s
1 3
Calculation took 6.23s
1 4
Calculation took 5.40s
1 5
Calculation took 5.16s
1 6
Calculation took 5.06s
1 7
Calculation took 4.98s
1 8
Calculation took 4.53s
1 9
Calculation took 3.79s
2 0
Calculation took 5.79s
2 1
Calculation took 6.01s
2 2
Calculation took 5.90s
2 3
Calculation took 5.58s
2 4
Calculation took 5.49s
2 5
Calculation took 5.53s
2 6
Calculation took 5.29s
2 7
Calculation took 4.53s
2 8
Calculation took 4.53s
2 9
Calculation took 4.55s
3 0
Calculation took 7.07s
3 1
Calculation took 6.27s
3 2
Calculation took 8.03s
3 3
Calculation took 6.97s
3 4
Calculation took 6.06s
3 5
Calculation took 5.73s
3 6
Calculation took 4.58s
3

In [12]:
figure(3)
clf()

vrange = 2

imshow(out, interpolation='none', cmap='seismic', vmin=-vrange, vmax=vrange, \
       extent=(xstart-0.5*((xstop-xstart)/(xdim-1)), xstop+0.5*((xstop-xstart)/(xdim-1)), \
               ystop+0.5*((ystop-ystart)/(ydim-1)), ystart-0.5*((ystop-ystart)/(ydim-1))),\
      aspect=(xstop-xstart)/(ystop-ystart))
       
colorbar()

xlabel('analyte concentration')
ylabel('capture concentration')
title('log10(protein conc) = %0.2f'%(pconc))

tight_layout()
draw()

savefig('heatmaps\\protein_conc-zoom_%0.2f.pdf'%(pconc))


#other things

In [106]:
t = 600

cap = result[t,:].reshape(-1, species)[:,cap_i]
prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
prot = result[t,:].reshape(-1, species)[:,prot_i] 

fig = figure(2)
clf()

#plot(cap, 'o-', label='cap')
plot(prot_cap, '--', label='prot-cap')
plot(prot, 'o-', label='prot')
plot(cap, 'o-', label='cap')

a = xlabel('test')

legend()

In [102]:
figure(1, figsize=(10, 8))
clf()

plot(volumes, result[:,prot_i-species], label='Coumarin-estradiol')
#plot(cvs, result[:,c_i-species])
#plot(cvs, result[:,cdp_i-species])
#plot(volumes, result[:,cdc_i-species], label='cdc')
#plot(volumes, result[:,cd_i-species], label='cd')
#plot(volumes, result[:,cdp_i-species], label='cdp')
grid()
legend()
#ylim((-0.000001, 0.00004))
xlabel('Volume / ml')
ylabel('Concentration')

savefig('example2.pdf')

#nondimensionalization

In [6]:
species=5
prot_i, cap_i, anal_i, prot_cap_i, prot_anal_i = range(0,species)
def f2(y, t, plate_flow, k_capture_on, k_capture_off, k_analyte_on, k_analyte_off):
    #get the input vector and reshape it into an interpretable array
    ar_y = y.reshape(-1, species)
    #array that we'll return with derivatives
    out = zeros(ar_y.shape)
       
    #get vectors for the concentration of each species
    prot = ar_y[:,prot_i]
    cap = ar_y[:,cap_i]
    anal = ar_y[:,anal_i]
    prot_cap = ar_y[:,prot_cap_i]
    prot_anal = ar_y[:,prot_anal_i]    
    
    #equilibrate each plate
    out[:,prot_i] = k_capture_off*prot_cap - k_capture_on*prot*cap + k_analyte_off*prot_anal - k_analyte_on*prot*anal
    out[:,anal_i] = k_analyte_off*prot_anal - k_analyte_on*prot*anal
    out[:,cap_i] = k_capture_off*prot_cap - k_capture_on*prot*cap
    out[:,prot_anal_i] = -(k_analyte_off*prot_anal - k_analyte_on*prot*anal)
    out[:,prot_cap_i] = -(k_capture_off*prot_cap - k_capture_on*prot*cap)  
    
    #flow between plates
    for key in [prot_i, anal_i, prot_anal_i]:
        additional = hstack((0, ar_y[:,key]))
        leaving = - plate_flow*ar_y[:,key]
        coming = plate_flow*additional[0:-1] 
        leaving[50:]=0
        coming[51:]=0
        out[:,key] = out[:,key] + leaving + coming
    
    return out.reshape(-1,1)[:,0]

In [7]:
out=[]
kfrange = range(0, 6)
for kinetic_factor in kfrange:
    kinetic_factor = 10**kinetic_factor
    npts=1000
    protein_start=10**-10.0
    anal_start=10**-8.0
    cap_start=10**-5.0
    kanal=10**10.0
    kcap=10**8.0
    #kinetic_factor = 1

    kanaloff = kinetic_factor
    kanalon = kanal*kanaloff
    kcapoff = kinetic_factor
    kcapon = kcap*kcapoff

    #nondimensionalize
    xc = protein_start
    tc = 1.0
    protein_start = protein_start / xc
    anal_start = anal_start / xc
    cap_start = cap_start / xc
    kanalon, kanaloff = kanalon*xc*tc, kanaloff*tc
    kcapon, kcapoff = kcapon*xc*tc, kcapoff*tc


    #global parameters that we want to keep constant for all simulations
    width, length, thickness = 10.0, 50.0/2.0, 0.15#units of mm
    porosity = 0.7
    col_vol = porosity*width*length*thickness / 1000.0 #ml
    #print 'Column volume is %0.2f ul'%(col_vol*1000)

    flow = 0.01 #ml/min
    flow = flow/60.0 #ml/s
    plates = 51
    plate_vol = col_vol / plates
    plate_flow = flow/plate_vol
    #print 'Plate volume is %0.2f ul'%(plate_vol*1000)
    volumes = linspace(0, col_vol*2.0, npts)
    times = volumes / flow
    cvs = volumes / col_vol


    col = zeros((plates, species))
    col[0, prot_i] = protein_start
    col[0, anal_i] = anal_start
    col[40, cap_i] = cap_start

    linear_col = col.reshape(-1,1)

    start = time.time()
    result = odeint(f2, linear_col[:,0], times, args=(plate_flow, kcapon, kcapoff, kanalon, kanaloff),\
                   printmessg=False)

    out.append(result)
    print "Calculation took %0.2fs"%(time.time()-start)

Calculation took 2.34s
Calculation took 4.15s
Calculation took 4.62s
Calculation took 5.10s
Calculation took 5.51s
Calculation took 5.00s


In [8]:
figure(3)
clf()
for i,v in enumerate(kfrange):
    t = -1
    result = out[i]
    cap = result[t,:].reshape(-1, species)[:,cap_i]
    prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
    prot = result[t,:].reshape(-1, species)[:,prot_i] 
    anal = result[t,:].reshape(-1, species)[:,anal_i] 
    prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i] 
    allprot = prot_cap+prot+prot_anal
    first_spot, second_spot = abs(allprot[40]), abs(allprot[50])
    
    plot(v, first_spot/second_spot, 'o')
grid()

In [248]:
#animation of the setup
t = -1

cap = result[t,:].reshape(-1, species)[:,cap_i]
prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i]
prot = result[t,:].reshape(-1, species)[:,prot_i] 
anal = result[t,:].reshape(-1, species)[:,anal_i] 
prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i] 

fig = figure(1, figsize=(10, 5))
clf()

#plot(cap, 'o-', label='cap')
prot_cap_line, = plot(prot_cap, '-', label='prot-cap')
prot_line, = plot(prot, '-', label='prot')
#anal_line, = plot(anal, '-', label='anal')
prot_anal_line, = plot(prot_anal, label='prot_anal')
legend(loc='upper left')

#ylim((-0.00001, 0.1))
grid()

if False:
    def animate(i):
        i=int(i)
        prot_cap_line.set_ydata(result[i,:].reshape(-1, species)[:,prot_cap_i])
        prot_line.set_ydata(result[i,:].reshape(-1, species)[:,prot_i])
        #anal_line.set_ydata(result[i,:].reshape(-1, species)[:,anal_i])
        prot_anal_line.set_ydata(result[i,:].reshape(-1, species)[:,prot_anal_i])
        #return prot_cap_line,prot_line,

    ani = animation.FuncAnimation(fig, animate, linspace(0, npts-1, 200),
        interval=10, blit=False, repeat=False)

show()

In [213]:
#check equilibrium
#t = 50

cap = result[t,:].reshape(-1, species)[:,cap_i] * xc
prot_cap = result[t,:].reshape(-1, species)[:,prot_cap_i] * xc
prot = result[t,:].reshape(-1, species)[:,prot_i]  * xc
anal = result[t,:].reshape(-1, species)[:,anal_i]  * xc
prot_anal = result[t,:].reshape(-1, species)[:,prot_anal_i]  * xc


figure(2)
clf()
plot(log10(prot_cap / (prot*cap)), 'o')
plot(log10(prot_anal / (prot*anal)), 'o')
draw()

C:\Users\rkwant\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipython-3.0.0-py2.7.egg\IPython\kernel\__main__.py:13: RuntimeWarning: invalid value encountered in true_divide
